In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import tensorflow as tf
from sklearn import preprocessing

In [2]:
# X = np.array([[0,0,1],\
#               [0,1,1],\
#               [1,0,1],\
#               [1,1,1]], dtype=np.float32)#1 appended in the last column is the bias

# t = { #dictionary for getting both the target logic values and the correlated string 
#     "AND":  np.array([[0], 
#                       [0],
#                       [0],
#                       [1]], dtype=np.float32),
    
#     "NAND": np.array([[1],
#                       [1],
#                       [1],
#                       [0]], dtype=np.float32),
    
#     "OR":   np.array([[0], 
#                       [1],
#                       [1],
#                       [1]], dtype=np.float32),
    
#     "NOR":  np.array([[1],
#                       [1],
#                       [1],
#                       [0]], dtype=np.float32),
    
#     "XOR":  np.array([[0],
#                       [1],
#                       [1],
#                       [0]], dtype=np.float32) }

In [3]:
X = np.array([[0,0,1],\
              [0,1,1],\
              [1,0,1],\
              [1,1,1]], dtype=np.float32)#1 appended in the last column is the bias

t = { #dictionary for getting both the target logic values and the correlated string 
    "AND":  np.array([[-1], 
                      [-1],
                      [-1],
                      [1]], dtype=np.float32),
    
    "NAND": np.array([[1],
                      [1],
                      [1],
                      [-1]], dtype=np.float32),
    
    "OR":   np.array([[-1], 
                      [1],
                      [1],
                      [1]], dtype=np.float32),
    
    "NOR":  np.array([[1],
                      [1],
                      [1],
                      [-1]], dtype=np.float32),
    
    "XOR":  np.array([[-1],
                      [1],
                      [1],
                      [-1]], dtype=np.float32) }

### function definitions
y_pred, online_perceptron are correct. Not sure about cost.
The nested for loop is terrible, I know. It was a required part of the implementation

In [4]:
# X is 4 x3 and w is 3x1 
def cost(y,x,w):
    return y.T@X@w

def online_perceptron(X, t, rho, epochs, gate_name=""):
    y = np.array([0, 0, 0, 0], dtype=np.float32)
    w_k = np.array([.5, .5, .5], dtype=np.float32)
    converged = False
    
    time_to_show = np.round(np.linspace(0,epochs, 40))
    
    k = 0
    
    for i in range(epochs):
        for j in range(len(X)):
             #zeros in [w_k, 0,...] is so that function selects the right w vector
            y[j] = y_pred(X[j,:], [w_k, 0, 0, 0])
            err = t[j] - y[j]
            w_k = w_k + rho*err*X[j,:]
            
        if len(y.shape) < 2: #make it a column vector if it is a flat array
            y = y.reshape(y.shape[0],1)
        if np.array_equal(y,t):
            converged = True
            break
            
#         if i == time_to_show[k]:
#             cost_w = cost(y, X, w_k.reshape(w_k.shape[0],1))
#             print("cost: " + str(cost_w))
#             k +=1
    
    w_pkg = [w_k.reshape(w_k.shape[0],1), converged, i+1, epochs, rho]
#     print("TRAIN PREDICTION >>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
#     y_pred(X, w_pkg,gate_name=gate_name,\
#            use_step_prediction=False, print_result=True)    
#     print("= end of perceptron ============================")
    
    return w_pkg

def y_pred(X, w_pckg, gate_name="No gate loaded",\
           use_step_prediction=False, print_result=False):
    # w_pckg[0]: w vector
    # w_pckg[1]: converged or not (True or False)
    # w_pckg[2]: no. of iterations to converge
    # w_pckg[3]: max no. of iterations
    # w_pckg[4]: learning rate
    y = X @ w_pckg[0] # 0 or 1 for output

    y = np.sign(y)
    
    if print_result:
        print(gate_name+ ":")
        print(np.heaviside(y.reshape(y.shape[0],1),0))
        if w_pckg[3]:
            print(str(w_pckg[2]) + " iterations to convergence")
        else:
            print("did not converge")
        print(str(w_pckg[3]) + " (max iterations)")
        print(str(w_pckg[4]) + " (learning rate)")
    return y

In [5]:
#runs for loops for all logic gate within the dictionary t
# for one learning rate AND a one no. of epochs
def predict_percept_all_gates(X, t, rho, epochs):
    
    w_pkg = {}
    for i in t:     # online_perceptron(X, t, rho, epochs, gate_name="")
        w_pkg[i] = online_perceptron(X, t[i], rho, epochs, gate_name=i)
    
    y_p = {}
    k = 0

    for i in t: #
#         y_pred(X, w_pckg, gate_name="No gate loaded", use_step_prediction=False, print_result=False):
        y_p[i] = y_pred(X, w_pkg[i], gate_name=i, print_result=True)
        print()
        k += 1
        
    return [y_p, w_pkg]

In [6]:
y = {}

# y: dictionary; to access members, use an element of the list RHO
# e.g. y[RHO[0]] == [y_pred, w_pkg], a list with a dict y_pred AND a dict the w_pkg 
#                                  y_pred: dict with 5 elements (indeces are the logic gates' names, capital letters)
#                                          each one a logic gate's prediction
#                                  w_pkg: dict with 5 elements (same indexing as y_pred)
#                                          each one a list with the the following elements:
#                                               [w_k, no_of_iterations_to_convergence, epochs, converged, rho]
#                                               [3-element np array(w vect), scalar, scalar, scalar, scalar]

#learning reates
RHO = np.array([.0001,.001,.01,.01,.1,1], dtype=np.float32)
#epochs
iterations = [200, 2_000, 20_000, 40_000]

for i in range(len(RHO)):
    print("===== learning rate: " + str(RHO[i]) + " =============================")
    y[RHO[i]] = predict_percept_all_gates(X, t, RHO[i], iterations[3])

===== learning rate: 1e-04 =============================
AND:
[[0.]
 [0.]
 [0.]
 [1.]]
1390 iterations to convergence
40000 (max iterations)
1e-04 (learning rate)

NAND:
[[1.]
 [1.]
 [1.]
 [0.]]
2506 iterations to convergence
40000 (max iterations)
1e-04 (learning rate)

OR:
[[0.]
 [1.]
 [1.]
 [1.]]
2501 iterations to convergence
40000 (max iterations)
1e-04 (learning rate)

NOR:
[[1.]
 [1.]
 [1.]
 [0.]]
2506 iterations to convergence
40000 (max iterations)
1e-04 (learning rate)

XOR:
[[1.]
 [1.]
 [0.]
 [0.]]
40000 iterations to convergence
40000 (max iterations)
1e-04 (learning rate)

===== learning rate: 0.001 =============================
AND:
[[0.]
 [0.]
 [0.]
 [1.]]
140 iterations to convergence
40000 (max iterations)
0.001 (learning rate)

NAND:
[[1.]
 [1.]
 [1.]
 [0.]]
256 iterations to convergence
40000 (max iterations)
0.001 (learning rate)

OR:
[[0.]
 [1.]
 [1.]
 [1.]]
251 iterations to convergence
40000 (max iterations)
0.001 (learning rate)

NOR:
[[1.]
 [1.]
 [1.]
 [0.]]
25

In [11]:
# example on first rho, y_pred selected, XOR
np.heaviside(y[RHO[0]][0]["XOR"],0)

array([[1.],
       [1.],
       [0.],
       [0.]], dtype=float32)

NameError: name 'w1' is not defined